In [ ]:
import re
import datetime
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
from time import sleep

In [ ]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r %s |%s| %s%% %s' % (prefix, bar, percent, suffix), end="")
    if iteration == total: 
        print()

In [ ]:
affx = 'http://www.espn.com/nba/game?gameId='
output = []
num_match = 8000
end_web = 400860570
counts = 0
errors = 0  
for webs in range(end_web - num_match, end_web+1):
        urls = affx + str(webs)
        try:
            r = urllib.request.urlopen(urls).read()
            soup = BeautifulSoup(r, "lxml")
            info_note = soup.find_all("div", class_="game-info-note capacity")
            if len(info_note) > 1:
                attendance_group = re.search("Attendance:\s(\d*),*(\d+)", info_note[0].text)
                capacity_group = re.search("Capacity:\s(\d*),*(\d+)", info_note[1].text)
                date_origin = soup.findAll("span", attrs={"data-behavior":"date_time"})[0]["data-date"]
                team_group = re.search("(\w+)\svs.\s(\w+)\s", soup.title.text)

                date = datetime.datetime.strptime(date_origin, "%Y-%m-%dT%H:%MZ") \
                       + datetime.timedelta(hours=19)
                try :
                    string = soup.find_all("div", class_="caption-wrapper")[0].text
                    game_center = re.search("[^\\t\\n]+", string).group(0)
                except IndexError:
                    game_center = soup.find_all("div", class_="location-details")[0].span.text
                geo_location = re.search("[^\\t\\n]+", \
                                         soup.find_all("li", \
                                         class_="icon-font-before icon-location-solid-before") \
                                         [0].text).group(0)
                attendance = int(attendance_group.group(1) + attendance_group.group(2))
                capacity = int(capacity_group.group(1) + capacity_group.group(2))
                home = team_group.group(1)
                guest = team_group.group(2)
                output.append([date, game_center, geo_location, attendance, capacity, home, guest])
                sleep(2)
        except urllib.error.URLError:
            errors += 1
            print ("\r errors %d" %(errors), end="")
            print ()
        counts += 1
        printProgressBar(counts, num_match, prefix = 'Progress:', suffix = 'Complete', length = 10)
        print()

In [ ]:
part_1 = pd.DataFrame(output)
part_1 = part_1.rename(columns={0:"date", 1:"game_center", 2:"geo_location", 
                               3:"attendance", 4:"capacity", 5:"home", 6:"guest"})
part_1.to_csv("part_5.csv")